## Importing the Libraries

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
tf.__version__

'2.13.0'

## Importing the dataset

In [2]:
dataset = pd.read_csv("Bank.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [ ]:
# np.set_printoptions(threshold=np.inf)

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


## Label encoding for dichotomous column

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


## OneHotEncoding for the polytomous variable

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Splitting the dataset into the training set and test set

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building the ANN

In [7]:
# Initializing ANN
ann = tf.keras.models.Sequential()

# Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding the output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compiling the ANN
ann.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training the ANN

In [8]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.5526 - accuracy: 0.7096
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4566 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4382 - accuracy: 0.7966
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4248 - accuracy: 0.8009
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4116 - accuracy: 0.8142
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3981 - accuracy: 0.8276
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3853 - accuracy: 0.8359
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3758 - accuracy: 0.8410
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3693 - accuracy: 0.8421
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3659 - accura

## Making the predictions and evaluating the model

Using our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 99ms/step
[[False]]


## Predicting the test set results

In [9]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the confusion matrix

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1519   76]
 [ 204  201]]


0.86